In [2]:
import pandas as pd

file_path = "docs/quora_duplicate_questions.tsv"
df = pd.read_csv(file_path, sep="\t")
questions = set()
for _, row in df.iterrows():
    obj = row.to_dict()
    questions.add(obj["question1"][:512])
    questions.add(obj["question2"][:512])
    if len(questions) > 500:
        break

docs = list(questions)
print(docs[0])

Which is correct - 'Looking forward to speak with you' or 'Look forward to speak with you'?


In [4]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

connections.connect(uri="./milvus.db")

fields = [
    FieldSchema(
        name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
]
schema = CollectionSchema(fields=fields)

collection_name = "hybrid_demo"
if utility.has_collection(collection_name):
    Collection(collection_name).drop()
col = Collection(collection_name, schema, consistency_level="Strong")

sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", index_params=sparse_index)
dense_index = {
    "index_type": "AUTOINDEX",
    "metric_type": "IP"
}
col.create_index("dense_vector", index_params=dense_index)
col.load()

I0000 00:00:1745657640.662998 63945837 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
